In [10]:
import numpy as np
from sklearn.metrics import confusion_matrix

def manual_lca_predict(model, X):
    # pesos de clases
    pis = model.weights_             # shape (K,)

    # extraer parámetros del modelo de medición
    params = model._mm.get_parameters()
    # params es dict con llaves como 'pis' para Bernoulli/categorical
    # supongamos que 'pis' almacena las probabilidades p(x_j = 1 | C = k)
    thetas = np.array(params['pis'])  # shape (K, n_variables)

    K, J = thetas.shape
    N = X.shape[0]

    loglik = np.zeros((N, K))

    for k in range(K):
        pk = thetas[k]
        log_p = np.log(pk + 1e-12)
        log_1_p = np.log(1 - pk + 1e-12)
        loglik[:, k] = (X * log_p + (1 - X) * log_1_p).sum(axis=1) + np.log(pis[k] + 1e-12)

    m = loglik.max(axis=1, keepdims=True)
    posterior = np.exp(loglik - m)
    posterior /= posterior.sum(axis=1, keepdims=True)

    classes_manual = posterior.argmax(axis=1)
    return classes_manual, posterior

def compare_with_stepmix(model, X):
    manual_pred, _ = manual_lca_predict(model, X)
    stepmix_pred = model.predict(X)
    cm = confusion_matrix(stepmix_pred, manual_pred)
    return cm, manual_pred, stepmix_pred


In [13]:
from stepmix.stepmix import StepMix

# Dataset binario de ejemplo
np.random.seed(42)
X = np.random.binomial(1, 0.5, size=(600, 4))

# Ajustar modelo LCA con 3 clases
model = StepMix(n_components=3, measurement="binary", n_steps=2)
model.fit(X)

# Comparar predict manual vs StepMix
cm, manual_pred, stepmix_pred = compare_with_stepmix(model, X)

print("Matriz de confusión entre StepMix predict y manual:")
print(cm)


C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Fitting StepMix...


Initializations (n_init) : 100%|██████████| 1/1 [00:00<00:00,  3.92it/s, max_LL=-1.66e+3, max_avg_LL=-2.77]

Matriz de confusión entre StepMix predict y manual:
[[150   0   0]
 [  0 177   0]
 [  0   0 273]]



C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\stepmix\stepmix.py:968: ConvergenceWarning: Initializations did not converge. Try different init parameters, or increase max_iter, abs_tol, rel_tol or check for degenerate data.
  warnings.warn(
C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


In [42]:
params["measurement"]

{'pis': array([[0.26166926, 0.38707557, 0.49598936, 0.64120047],
        [0.79214252, 0.74874507, 0.41970643, 0.94255631],
        [0.63676697, 0.52458225, 0.52405434, 0.22758711]])}

In [45]:
params["structural"]

{'beta': array([[ 0.26543352,  0.08912555,  0.0056286 ],
        [-0.58299218, -0.04808801,  0.00908393],
        [ 0.31797345, -0.0420023 , -0.0148835 ]])}

In [46]:
import numpy as np
from sklearn.metrics import confusion_matrix

def manual_lca_predict(model, X, Z=None):
    """
    Replica manualmente predict() para StepMix con:
    - medición Bernoulli
    - modelo estructural multinomial logit (covariates)

    Compatible con tu versión REAL del modelo:
        params = model.get_parameters()
        keys: ['weights', 'measurement', 'measurement_in', 'structural', 'structural_in']
    """

    params = model.get_parameters()

    # -------------------------------
    # 1. Probabilidades de clase π_k
    # -------------------------------
    pis = params["weights"]                      # shape (K,)

    # -------------------------------
    # 2. Medición Bernoulli p[k,j]
    # -------------------------------
    bern_p = params["measurement"]["pis"]          # shape (K, J)

    # log p(x[j] | class k) sumando sobre j
    log_px_given_k = (
        X[:, None, :] * np.log(bern_p + 1e-12) +
        (1 - X[:, None, :]) * np.log(1 - bern_p + 1e-12)
    ).sum(axis=2)  # (N, K)

    # -------------------------------
    # 3. Modelo estructural (covariables)
    # -------------------------------
    if Z is None:
        # Sin covariables → prior π_k fijo
        log_prior = np.log(pis + 1e-12)[None, :]

    else:
        # Con covariables → multinomial logit
        structural = params["structural"]

        coef = structural["beta"]         # shape (K, p)
        intercept = structural["intercept"]  # shape (K,)

        logits = Z @ coef.T + intercept   # shape (N, K)
        logits -= logits.max(axis=1, keepdims=True)
        class_probs = np.exp(logits)
        class_probs /= class_probs.sum(axis=1, keepdims=True)

        log_prior = np.log(class_probs + 1e-12)

    # -------------------------------
    # 4. Posteriores ∝ log likelihood + log prior
    # -------------------------------
    log_gamma = log_px_given_k + log_prior
    log_gamma -= log_gamma.max(axis=1, keepdims=True)

    gamma = np.exp(log_gamma)
    gamma /= gamma.sum(axis=1, keepdims=True)

    # predicción MAP
    pred = gamma.argmax(axis=1)

    return pred, gamma


def compare_with_stepmix(model, X, Z=None):
    manual_pred, _ = manual_lca_predict(model, X, Z)
    official_pred = model.predict(X, Z)
    cm = confusion_matrix(official_pred, manual_pred)
    return cm, manual_pred, official_pred


In [84]:
params = model.get_parameters()
structural = params["structural"]
model.structural_in_ == 1

False

In [88]:
model.structural_in_

2

In [92]:
import numpy as np
from sklearn.metrics import confusion_matrix

def manual_lca_predict(model, X, Z=None):
    """
    Replica manualmente el predict de StepMix para un modelo LCA binario,
    con o sin covariables Z.
    """

    params = model.get_parameters()

    # ----- 1. Pesos por clase (peta prior)
    weights = params["weights"]   # shape (K,)

    # ----- 2. Parámetros de medición (Bernoulli)
    measurement = params["measurement"]  # dict con claves 'proba'
    p = measurement["pis"]             # shape (K, D)

    # ----- 3. Estructural (si hay covariables)
    structural = params["structural"]
    has_covariates = (model.structural_in_ >= 1)

    if has_covariates:
        gamma = structural["beta"]     # shape (K, p+1)
        intercept = gamma[:, 0]         # shape (K,)
        beta = gamma[:, 1:]             # shape (K, p)
        
        logits = intercept + Z @ beta.T
        logits -= logits.max(axis=1, keepdims=True)
        class_prior = np.exp(logits)
        class_prior /= class_prior.sum(axis=1, keepdims=True)  # shape (N, K)

    else:
        class_prior = np.repeat(weights[np.newaxis, :], X.shape[0], axis=0)

    # ----- 4. Likelihood de items Bernoulli
    eps = 1e-12
    ll = X[:, None, :] * np.log(p + eps) + (1 - X[:, None, :]) * np.log(1 - p + eps)
    ll = ll.sum(axis=2)  # shape (N, K)

    # ----- 5. Posteriores (responsabilidades)
    log_post = np.log(class_prior + eps) + ll
    log_post -= log_post.max(axis=1, keepdims=True)
    post = np.exp(log_post)
    post /= post.sum(axis=1, keepdims=True)

    pred = np.argmax(post, axis=1)
    return pred, post


In [93]:
import numpy as np
from stepmix.stepmix import StepMix

# Datos de ejemplo
np.random.seed(0)
N = 5000
J = 10
X = np.random.binomial(1, 0.5, size=(N, J))
Z = np.random.normal(size=(N, 2))

# Modelo con covariables
model = StepMix(
    n_components=5,
    measurement="bernoulli",
    structural="covariate",
    n_steps=3
)

model.fit(X, Z)

# Comparar predicciones
cm, manual_pred, auto_pred = compare_with_stepmix(model, X, Z)

print("Matriz de confusión manual vs StepMix:")
print(cm)


C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Fitting StepMix...


Initializations (n_init) : 100%|██████████| 1/1 [00:00<00:00,  1.10it/s, max_LL=-3.46e+4, max_avg_LL=-6.92]

Matriz de confusión manual vs StepMix:
[[ 164    0    0    0    0]
 [   0 2486    0    0    0]
 [   0    0 1262    0    0]
 [   0    0    0  796    0]
 [   0    0    0    0  292]]



C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\stepmix\stepmix.py:968: ConvergenceWarning: Initializations did not converge. Try different init parameters, or increase max_iter, abs_tol, rel_tol or check for degenerate data.
  warnings.warn(
C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.val

In [62]:
params["structural"]

{'beta': array([[ 0.26543352,  0.08912555,  0.0056286 ],
        [-0.58299218, -0.04808801,  0.00908393],
        [ 0.31797345, -0.0420023 , -0.0148835 ]])}

In [66]:
params["intercept"]

KeyError: 'intercept'

In [63]:
model.structural_.intercept_

AttributeError: 'StepMix' object has no attribute 'structural_'

In [61]:
import numpy as np
from stepmix.stepmix import StepMix
from sklearn.metrics import confusion_matrix

# ============
# 1. Función predict manual
# ============

def manual_lca_predict(model, X, Z=None):
    """
    Replica manualmente el predict de StepMix para un modelo LCA binario,
    con o sin covariables Z.
    """

    params = model.get_parameters()

    # Pesos iniciales
    weights = params["weights"]   # shape (K,)

    # Measurement Bernoulli
    measurement = params["measurement"]
    p = measurement["pis"]      # shape (K, D)

    # Structural model
    structural = params["structural"]
    has_covariates = (model.structural_in_ == 1)

    if has_covariates:
        # gamma: (K, p)
        # Como Z ya incluye intercepto, gamma ya incluye ese coeficiente
        gamma = structural["gamma"]   # shape (K, p)

        logits = Z @ gamma.T          # shape (N, K)
        logits -= logits.max(axis=1, keepdims=True)
        class_prior = np.exp(logits)
        class_prior /= class_prior.sum(axis=1, keepdims=True)

    else:
        class_prior = np.repeat(weights[np.newaxis, :], X.shape[0], axis=0)

    # Likelihood Bernoulli
    eps = 1e-12
    ll = X[:, None, :] * np.log(p + eps) + (1 - X[:, None, :]) * np.log(1 - p + eps)
    ll = ll.sum(axis=2)

    # Posterior
    log_post = np.log(class_prior + eps) + ll
    log_post -= log_post.max(axis=1, keepdims=True)
    post = np.exp(log_post)
    post /= post.sum(axis=1, keepdims=True)

    pred = np.argmax(post, axis=1)
    return pred, post

# ============
# 2. Comparación con StepMix
# ============

def compare_with_stepmix(model, X, Z=None):
    manual_pred, _ = manual_lca_predict(model, X, Z)
    official_pred = model.predict(X, Z)
    cm = confusion_matrix(official_pred, manual_pred)
    return cm, manual_pred, official_pred


# ============
# 3. Ejemplo con intercepto explícito en Z
# ============

np.random.seed(123)
N = 200

# Covariables
Z_raw = np.random.normal(size=(N, 2))             # dos covariables
Intercept = np.ones((N, 1))                       # <-- intercepto manual
Z = np.hstack([Intercept, Z_raw])                 # Z = [1, Z1, Z2]

# Variables Y ~ Bernoulli
X = np.random.binomial(1, 0.5, size=(N, 5))

# Modelo StepMix
model = StepMix(
    n_components=3,
    measurement="bernoulli",
    structural="covariate",   # usa Z
    n_steps=1,
    max_iter=400,
    random_state=123
)

model.fit(X, Z)

# Comparación
cm, manual_pred, auto_pred = compare_with_stepmix(model, X, Z)

print("Matriz de confusión manual vs StepMix:")
print(cm)


C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was

Fitting StepMix...


Initializations (n_init) : 100%|██████████| 1/1 [00:00<00:00,  8.51it/s, max_LL=-677, max_avg_LL=-3.39]

Matriz de confusión manual vs StepMix:
[[46 34  1]
 [32 75  2]
 [ 0  5  5]]



C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' wa

In [57]:

# ============
# 3. Ejemplo con intercepto explícito en Z
# ============

np.random.seed(123)
N = 200

# Covariables
Z_raw = np.random.normal(size=(N, 2))             # dos covariables
Intercept = np.ones((N, 1))                       # <-- intercepto manual
Z = np.hstack([Intercept, Z_raw])                 # Z = [1, Z1, Z2]

# Variables Y ~ Bernoulli
X = np.random.binomial(1, 0.5, size=(N, 5))

# Modelo StepMix
model = StepMix(
    n_components=3,
    measurement="bernoulli",
    structural="covariate",   # usa Z
    n_steps=1,
    max_iter=400,
    random_state=123
)

model.fit(X, Z)

# Comparación
cm, manual_pred, auto_pred = compare_with_stepmix(model, X, Z)

print("Matriz de confusión manual vs StepMix:")
print(cm)

C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was

Fitting StepMix...


Initializations (n_init) : 100%|██████████| 1/1 [00:00<00:00,  6.32it/s, max_LL=-677, max_avg_LL=-3.39]

Matriz de confusión manual vs StepMix:
[[46 34  1]
 [32 75  2]
 [ 0  5  5]]



C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' wa

In [38]:
model = StepMix(
    n_components=3,
    measurement="bernoulli",
    structural="covariate",
    n_steps=3
)

model.fit(X, Z)

#print("Atributos del modelo:")
#for a in dir(model):
#    if not a.startswith("_"):
#        print(a)
        
params = model.get_parameters()
print(params.keys())

# Y también:
for k, v in params.items():
    print(k, type(v))

C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Fitting StepMix...


Initializations (n_init) : 100%|██████████| 1/1 [00:00<00:00,  4.46it/s, max_LL=-1.38e+3, max_avg_LL=-2.77]

dict_keys(['weights', 'measurement', 'measurement_in', 'structural', 'structural_in'])
weights <class 'numpy.ndarray'>
measurement <class 'dict'>
measurement_in <class 'int'>
structural <class 'dict'>
structural_in <class 'int'>



C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\stepmix\stepmix.py:968: ConvergenceWarning: Initializations did not converge. Try different init parameters, or increase max_iter, abs_tol, rel_tol or check for degenerate data.
  warnings.warn(
C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.val

In [67]:
model = StepMix(
    n_components=3,
    measurement="bernoulli",
    structural="covariate",
    n_steps=3
)

model.fit(X, Z)

print("Atributos del modelo:")
for a in dir(model):
    if not a.startswith("_"):
        print(a)

C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Fitting StepMix...


Initializations (n_init) : 100%|██████████| 1/1 [00:00<00:00,  6.92it/s, max_LL=-689, max_avg_LL=-3.44]

Atributos del modelo:
abs_tol
aic
assignment
bic
bootstrap
bootstrap_stats
caic
converged_
correction
em
entropy
fit
get_cw_df
get_metadata_routing
get_mm_df
get_parameters
get_parameters_df
get_params
get_sm_df
init_params
log_resp_
lower_bound_
lower_bound_buffer_
m_step_structural
max_iter
measurement
measurement_in_
measurement_params
n_components
n_features_in_
n_init
n_iter_
n_parameters
n_steps
param_buffer_
permute_classes
predict
predict_Y
predict_class
predict_proba
predict_proba_Y
predict_proba_class
progress_bar
random_state
rel_tol
relative_entropy
report
sabic
sample
save_param_init
score
set_fit_request
set_parameters
set_params
set_score_request
structural
structural_in_
structural_params
verbose
weights_
x_names_
y_names_



C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\stepmix\stepmix.py:968: ConvergenceWarning: Initializations did not converge. Try different init parameters, or increase max_iter, abs_tol, rel_tol or check for degenerate data.
  warnings.warn(
C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.val

In [68]:
model = StepMix(
    n_components=3,
    measurement="bernoulli",
    structural="covariate",
    n_steps=3
)

model.fit(X, Z)

C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


Fitting StepMix...


Initializations (n_init) : 100%|██████████| 1/1 [00:00<00:00,  6.27it/s, max_LL=-689, max_avg_LL=-3.44]
C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\stepmix\stepmix.py:968: ConvergenceWarning: Initializations did not converge. Try different init parameters, or increase max_iter, abs_tol, rel_tol or check for degenerate data.
  warnings.warn(
C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\sklearn\base.py:474: FutureWarning:

StepMix(n_components=3, n_steps=3, structural='covariate')

In [72]:
model.get_sm_df()

C:\Users\sajlv\AppData\Roaming\Python\Python311\site-packages\stepmix\stepmix.py:1224: FutureWarning: The provided callable <function mean at 0x000001E2CF9E1080> is currently using DataFrameGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  return pd.pivot_table(


class_no                           0         1         2
model_name param variable                               
covariate  beta  feature_0 -0.347793  0.109779  0.239740
                 feature_1  0.026914  0.074214 -0.100186
                 feature_2  0.036540  0.025276 -0.061052
                 intercept -0.347292  0.108822  0.237274

In [75]:
model.get_parameters()["structural"]

{'beta': array([[-0.34729202, -0.34779284,  0.02691427,  0.03654036],
        [ 0.10882199,  0.10977946,  0.07421362,  0.02527605],
        [ 0.23727398,  0.23973986, -0.1001856 , -0.06105248]])}